In [1]:
import requests
import csv, re, os
import json
import pandas as pd
import numpy as np
from time import sleep
import random

In [2]:
import time

In [3]:
answers = pd.read_csv('weibo/weibo_box.csv',dtype=object)
savefile = 'weibo/weibo_comments_box.csv'
savefile_root = 'weibo/weibo_root_count_box.csv'

In [4]:
## check index!!!!!!
answers = answers.drop_duplicates('mid').reset_index()

In [16]:
with open('weibo/cookie.txt', 'r') as f:
    Cookie = f.readlines()[0]  # Read all lines in the file
    User_Agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Safari/605.1.15"
    headers = {
            'Cookie': Cookie,
            'User-Agent': User_Agent
        }

In [27]:
with open(savefile, 'a+') as fc, open(savefile_root, 'a+') as fr:
    header_fc = ['answer_id','answer_content','comment_id','created_time','comment_content','comment_upvote',
              'child_comments_count','author_info','author_gender','author_location','author_following','author_followers']
    writer_fc = csv.DictWriter(fc, delimiter=',', lineterminator='\n',fieldnames=header_fc)
    if not os.path.getsize(savefile):
        writer_fc.writeheader()

    header_fr = ['answer_id','root_comment']
    writer_fr = csv.DictWriter(fr, delimiter=',', lineterminator='\n',fieldnames=header_fr)
    if not os.path.getsize(savefile_root):
        writer_fr.writeheader()
        index_start = 0
    else:
        index_start = len(pd.read_csv(savefile_root))
    print(index_start)
    start_time = time.time()
    for index in range(index_start, len(answers)):
        try:
            row = answers.loc[index]
        except:
            continue
        page = 0
        root_comment = 0
        mid = row['mid']
        uid = row['author_uid']
        if not type(row['text']) == str:
            root_comment = -1
        elif row['text'].startswith('//@'):
            root_comment = -1
        elif type(row['comment']) == str:
            url = f'https://weibo.com/ajax/statuses/buildComments?is_reload=1&id={mid}&is_show_bulletin=2&is_mix=0&count=10&uid={uid}&fetch_level=0'
            while True:
                start_time = time.time()
                try:
                    res = json.loads(requests.get(url, headers=headers).text)
                except:
                    break
                print(time.time()-start_time)

                if res['ok'] == 0:
                    break

                comments = res['data']
                for d in comments:
                    comment_id = d['id']
                    comment_content = re.sub('<.+>', '', d['text'])
                    try:
                        author_location = d['source'].replace('来自','')
                    except:
                        author_location = 'None'
                    author_info = d['user']['profile_url']
                    author_gender = d['user']['gender']
                    author_following = d['user']['friends_count']
                    author_followers = d['user']['followers_count']
                    child_comments_count = d['total_number']
                    try:
                        comment_like = d['like_counts']
                    except:
                        comment_like = d['like_count']
                    writer_fc.writerow({'answer_id':row['mid'], 'answer_content':row['text'],'comment_id':comment_id,
                                     'created_time':d['created_at'],'comment_content':comment_content,'comment_upvote':comment_like,
                                     'child_comments_count':child_comments_count,'author_info':author_info,'author_gender':author_gender,
                                     'author_location':author_location,'author_following':author_following,'author_followers':author_followers})
                    fc.flush()
                root_comment += len(comments)
                max_id = str(res['max_id'])
                
                
                if max_id == '0' or len(comments)==0:
                    break
                url = f'https://weibo.com/ajax/statuses/buildComments?flow=0&is_reload=1&id={mid}&is_show_bulletin=2&is_mix=0&max_id={max_id}&count=20&uid={uid}&fetch_level=0'
                #sleep(random.uniform(2, 4))
                print(mid, len(comments), root_comment)
                
        else:
            root_comment = 0
        sleep(random.uniform(2, 4))
        writer_fr.writerow({'answer_id':row['mid'],'root_comment':root_comment})
        fr.flush()

116612
1.478484869003296
2.5754029750823975
1.68034029006958
1.7614052295684814
4629853242659095 16 16
2.422961950302124
1.8959519863128662
4629850856098850 17 17
1.4929187297821045
4629850856098850 16 33
1.4968681335449219
4629850856098850 13 46
1.9900758266448975
4629850856098850 9 55
1.6193959712982178
1.3781020641326904
1.268028974533081
1.4370739459991455
1.8068790435791016
1.6355280876159668
1.5115232467651367
1.7229466438293457
1.5154600143432617
1.4412391185760498
1.48250412940979
1.419233798980713
1.5026969909667969
2.1455790996551514
1.8883941173553467
1.4797499179840088
1.4957029819488525
1.2897329330444336
1.6336431503295898
1.3504886627197266
1.4664928913116455
1.4150362014770508
1.4731719493865967
1.5759119987487793
1.4034008979797363
1.5564351081848145
1.9119470119476318
1.479151964187622
1.7911601066589355
1.8567450046539307
4629653195072075 18 18
1.692060947418213
4629653195072075 14 32
1.562267780303955
4629653195072075 7 39
1.512359857559204
1.8428359031677246
2.2432